# Introduction

In [ ]:
# retrieve the sp code for shortest path calculation
!rm -rf sp && mkdir sp
!wget "https://github.com/UCB-CE170a/Fall2020/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/interface.py" -O sp/interface.py

# retrieve the road network and demand inputs
!rm -rf traffic_inputs && mkdir traffic_inputs
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_edges.csv" -O traffic_inputs/berkeley_edges.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/berkeley_nodes.csv" -O traffic_inputs/berkeley_nodes.csv
!wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2020/master/traffic_data/od_10pn.csv" -O traffic_inputs/od_10pn.csv

In [5]:
import pandas as pd
from sp import interface

In [21]:
# read network files
edges_df = pd.read_csv('traffic_inputs/berkeley_edges.csv')
display(edges_df.head(2))
nodes_df = pd.read_csv('traffic_inputs/berkeley_nodes.csv')
display(nodes_df.head(2))

# read travel demand (origin-destination) files
od_df = pd.read_csv('traffic_inputs/od_10pn.csv')
display(od_df.head(2))

### make road network graph
g = interface.from_dataframe(edges_df, 'start_node_id', 'end_node_id', 'fft')

### create dictionaries between node_id and link_id for quick lookup
endnode2link_dict = {(getattr(link, 'start_node_id'), getattr(link, 'end_node_id')): getattr(link, 'link_id') for link in edges_df.itertuples()}
link2endnode_dict = {getattr(link, 'link_id'): (getattr(link, 'start_node_id'), getattr(link, 'end_node_id')) for link in edges_df.itertuples()}

# define a helper function to retrieve and format the shortest path between an origin-destination pair
def get_path(origin, destin):
    """
    Retrieving the shortest path between a given origin-destination pair.

    Parameters:
    origin (int): origin node id
    destin (int): destination node id
    endnode2link_dict (dict): a dictionary that maps the start and end node to the link id

    Returns:
    route (list of int): the shortest path, represented by a list of ids of the links that form the path
    sp_dist (float): the shortest path distance
    """
    ### retrieve shortest path between the origin and destination nodes using Dijkstra's Algorithm
    sp = g.dijkstra(origin, destin)
    ### calculate the path distance
    sp_dist = sp.distance(destin)

    if sp_dist > 1e8:
        ### if path does not exist between the origin and destination, it will return a very high distance value
        route = []
    else:
        ### otherwise, a path is found. We will convert them into a list of links, where each element is the id of the link
        route = [endnode2link_dict[(start_node_id, end_node_id)] for (start_node_id, end_node_id) in sp.route(destin)]
    sp.clear()
    
    return route, sp_dist

,link_id,start_node_id,end_node_id,type,length,maxmph,lanes,capacity,fft,start_osmid,end_osmid,geometry
0,0,222,210,secondary,127.923367,25.0,2,3800,11.446276,53096087_m,53091866_m,"LINESTRING (-122.2690042 37.87749114999998, -1..."
1,1,177,222,secondary,135.477223,25.0,2,3800,12.122177,53096090_m,53096087_m,"LINESTRING (-122.269111 37.87842139999998, -12..."


,node_id,lon,lat,type,node_osmid
0,0,-122.273552,37.885382,real,53095436
1,1,-122.267467,37.903256,real,53058576


,origin_node_id,destin_node_id,origin_osmid,destin_osmid
0,91,237,100526670,vn_sink
1,91,237,100526670,vn_sink


Below is an example of computing the shortest path for one od pair, e.g., the path from node 91 to node 237.

- The `get_path()` function takes the input returns: (1) the shortest path between each od pair as a list, and (2) the shortest path distance. In the first return value, each list element denotes a road link by its id. For instance, in the example here, the path from node 91 and node 237 is `[747, 266, 198, 39, 41, 34, 110, 189, 18, 219, 827]`. Its first element `747` denotes a road link that connects node 91 and node 155.
- The `traffic_volume_dictionary` is a dictionary-type variable that we use to count the traffic volume on each link. This dictionary's keys are road ids and values are number of trips (i.e., traffic volumes) that use each link. Since the dictionary is initiated to be empty, we use a `try ... except ...` block to update it. If `try ...` fails, it means that the particular road link id has not been added to the dictionary yet, so we will move on to the `except ...` clause and create a new key in the dictionary corresponding to the link id.

In [26]:
### example of path calculation
origin = 91      ### the origin node id of a trip
destin = 237     ### the end node id of a trip
route, distance = get_path(origin, destin) ### get the shortest path between the origin and destination of the trip
print('The shortest path between {} and {} is {}'.format(origin, destin, route))
print('The path/route object is a {}, the first elment is {}'.format(type(route), route[0]))
print('The trip travel time is {:.2f} minutes.\n'.format(distance/60))

### example of traffic volume calculation
traffic_volume_dictionary = dict()
for path_link in route:
  try:
    traffic_volume_dictionary[path_link] += 1
  except KeyError:
    traffic_volume_dictionary[path_link] = 1

print( 'Link {}, connecting node {} and {}, has {} trip(s) passed through it (so far)'.format( 747, link2endnode_dict[747][0], link2endnode_dict[747][1], traffic_volume_dictionary[747] ) )

### example of converting dictionary to dataframe that contains traffic volume for each road link
traffic_volume_df = pd.DataFrame(list(traffic_volume_dictionary.items()), columns=['link_id', 'volume'])
traffic_volume_df.head()

The shortest path between 91 and 237 is [747, 266, 198, 39, 41, 34, 110, 189, 18, 219, 827]
The path/route object is a <class 'list'>, the first elment is 747
The trip travel time is 3.32 minutes.

Link 747, connecting node 91 and 155, has 1 trip(s) passed through it (so far)


,link_id,volume
0,747,1
1,266,1
2,198,1
3,39,1
4,41,1



Given the inputs (`edges_df`, `nodes_df`, `od_df`, `g`, and the `get_path` fuction),
- Task 1: calculate the path for each od pair
- Task 2: compute the number of trips that pass each link (i.e., traffic volume on this link)
- Task 3: save the result of Task 2 in a CSV file `quiz3_[your_name].csv`. The file should look like:

| link_id | volume      |
|---------|-------------|
| 0       | ...         |
| 1       | ...         |
| ...     | ...         |

Hints:
- Task 1: Loop through the `od` dictionary and use the `get_path()` function in the code block above to compute the path for each od pair. The `get_path()` function is reusable, so you just need to call it and no need to define it every time you compute the path.
- Task 2: Initiate an empty `trffic_volume_dictionary` outside of the loop that iterates through each od pair. The key of the dictionary should be link_id (e.g., `747`), and the value corresponding to each key denotes the traffic volume on this link. While iterating through the od pairs, whenever a link is being used (e.g., whenever a new path is computed, and `747` is in the path), update the corresponding dictionary value by adding one, to represent that there is one more trip using the link.
- Beware that links are directed, meaning that link with end nodes `(91, 155)` is different from link whose end nodes are `(155, 91)`.
- **The method in the hints above are not the most efficient. It is just one way to complete the tasks. If you are comfortable of other ways, e.g., using Pandas groupby and merge functions, please choose a method that works the best for you.**

In [ ]:
### YOUR CODE HERE ###
### create a dataframe that counts the traffic volume on each road link under the given travel demand
### name the output file as "quiz3_[your_name].csv"

your_name = '' ### your name here
traffic_volume_df.to_csv('quiz3_{}.csv'.format(your_name), index=False)